In [2]:
import requests
import cv2
import numpy as np
import os

ROBOFLOW_API_KEY = "qGaBcdFPu8o5nVzk6TVH"
MODEL_ID = "tooth-segmentation-ra8um"
VERSION_NUMBER = "4"
IMAGE_DIR = "D:\Pertugasan\lah wes skripsi bjir\periapical-photo-caries-detection-using-deep-learning\dataset\mix"
OUTPUT_DIR = "D:\Pertugasan\lah wes skripsi bjir\periapical-photo-caries-detection-using-deep-learning\segmented_teeth"

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Roboflow API URL
ROBOFLOW_URL = f"https://detect.roboflow.com/{MODEL_ID}/{VERSION_NUMBER}?api_key={ROBOFLOW_API_KEY}"

def segment_image(image_path):
    """Send an image to Roboflow and return detected tooth regions."""
    with open(image_path, "rb") as img_file:
        response = requests.post(
            ROBOFLOW_URL, 
            files={"file": img_file},
            data={"confidence": 40, "overlap": 30}  # Adjust confidence & overlap as needed
        )

    response_json = response.json()
    
    if "predictions" not in response_json:
        print(f"Error processing {image_path}: {response_json}")
        return []

    return response_json["predictions"]

def save_cropped_teeth(image_path, predictions):
    """Crop and save individual teeth from an image based on predictions."""
    image = cv2.imread(image_path)
    image_name = os.path.basename(image_path).split('.')[0]

    for i, pred in enumerate(predictions):
        x, y, width, height = int(pred["x"]), int(pred["y"]), int(pred["width"]), int(pred["height"])

        # Extract bounding box
        x1, y1 = max(0, x - width // 2), max(0, y - height // 2)
        x2, y2 = min(image.shape[1], x + width // 2), min(image.shape[0], y + height // 2)

        tooth_crop = image[y1:y2, x1:x2]

        # Save the extracted tooth
        tooth_filename = os.path.join(OUTPUT_DIR, f"{image_name}_tooth_{i}.png")
        cv2.imwrite(tooth_filename, tooth_crop)
        print(f"Saved: {tooth_filename}")

def process_dataset():
    """Process all images in the dataset directory."""
    for filename in os.listdir(IMAGE_DIR):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(IMAGE_DIR, filename)
            print(f"Processing: {image_path}")

            predictions = segment_image(image_path)
            if predictions:
                save_cropped_teeth(image_path, predictions)

if __name__ == "__main__":
    process_dataset()


Processing: D:\Pertugasan\lah wes skripsi bjir\periapical-photo-caries-detection-using-deep-learning\dataset\mix\caries-1-_png.rf.84b6d403e60d07ac904560e9bd1b4a1f.jpg
Saved: D:\Pertugasan\lah wes skripsi bjir\periapical-photo-caries-detection-using-deep-learning\segmented_teeth\caries-1-_png_tooth_0.png
Saved: D:\Pertugasan\lah wes skripsi bjir\periapical-photo-caries-detection-using-deep-learning\segmented_teeth\caries-1-_png_tooth_1.png
Saved: D:\Pertugasan\lah wes skripsi bjir\periapical-photo-caries-detection-using-deep-learning\segmented_teeth\caries-1-_png_tooth_2.png
Processing: D:\Pertugasan\lah wes skripsi bjir\periapical-photo-caries-detection-using-deep-learning\dataset\mix\caries-10-_png.rf.afd723b6483c38df2385b9b332589016.jpg
Saved: D:\Pertugasan\lah wes skripsi bjir\periapical-photo-caries-detection-using-deep-learning\segmented_teeth\caries-10-_png_tooth_0.png
Saved: D:\Pertugasan\lah wes skripsi bjir\periapical-photo-caries-detection-using-deep-learning\segmented_teeth\